In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
#from config import weather_api_key
#from config import g_key
from citipy import citipy
import csv as atlantic_working

In [9]:
# Import API key
#from config import weather_api_key

In [12]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [13]:
awdf = pd.read_csv('atlanticworking.csv')
awdf['Latitude'].head()
#awdf['Longitude'].head()

FileNotFoundError: [Errno 2] No such file or directory: 'atlanticworking.csv'

In [25]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)

In [11]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [45]:
awdf['Latitude']=df_city['lat']
awdf['Longitude']=df_city['lng']
df_city.count()

City    6500
lat     6500
lng     6500
dtype: int64

In [32]:
awdf.head()

,ID,Name,Year,Month,Date,Hour,Event,Status,Latitude,A low that is neither of the 3 cyclone variantsngitude,...,A low that is neither of the 3 cyclone variantsw Wind SW,A low that is neither of the 3 cyclone variantsw Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,1851,6,25,0,,Hurricane Intensity > 64 knoTropical Storm 34...,30.805139,94.8W,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,1851,6,25,600,,Hurricane Intensity > 64 knoTropical Storm 34...,29.120529,95.4W,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,1851,6,25,1200,,Hurricane Intensity > 64 knoTropical Storm 34...,31.006394,96.0W,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,1851,6,25,1800,,Hurricane Intensity > 64 knoTropical Storm 34...,29.329358,96.5W,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,1851,6,25,2100,Landfall,Hurricane Intensity > 64 knoTropical Storm 34...,30.184307,96.8W,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [12]:
#check length of cities
len(list(df_city["City"].unique()))

113

In [17]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['huntsville',
 'hondo',
 'san+angelo',
 'galveston',
 'vidor',
 'harker+heights',
 'orange',
 'leon+valley',
 'taylor',
 'kirby',
 'lufkin',
 'temple',
 'pleasanton',
 'brownwood',
 'beaumont',
 'lakeway',
 'bryan',
 'copperas+cove',
 'brenham',
 'fredericksburg',
 'liberty',
 'bellmead',
 'sulphur',
 'gatesville',
 'katy',
 'nacogdoches',
 'el+campo',
 'uvalde',
 'kerrville',
 'victoria',
 'jacksonville',
 'lumberton',
 'freeport',
 'port+arthur',
 'kyle',
 'seguin',
 'palestine',
 'conroe',
 'lockhart',
 'natchitoches',
 'robinson',
 'belton',
 'san+marcos',
 'friendswood',
 'baytown',
 'rosenberg',
 'the+woodlands',
 'la+marque',
 'aldine',
 'beeville',
 'college+station',
 'pasadena',
 'port+lavaca',
 'bellaire',
 'alvin',
 'canyon+lake',
 'nederland',
 'austin',
 'spring',
 'converse',
 'wharton',
 'waco',
 'leander',
 'pearland',
 'san+antonio',
 'woodway',
 'del+rio',
 'texas+city',
 'missouri+city',
 'live+oak',
 'lake+jackson',
 'eagle+pass',
 'killeen',
 'pflugerville',
 'cl

In [13]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [14]:
base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
temp=[]
wind_speed=[]
pressure=[]
city_name=[]

for city in target_cities[:4]:
    query_url=f"{base_current_url}appid={weather_api_key}&q={city},US&units=imperial"
    req = requests.get(query_url)
    response1=req.json()
    latitude=response1["coord"]["lat"]
    longitude=response1["coord"]["lon"]

    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        temp.append(hist_json["current"]["temp"])
        pressure.append(hist_json["current"]["pressure"])
        wind_speed.append(hist_json["current"]["wind_speed"])
        city_name.append(response1['name'])
        print(historical_url)

NameError: name 'target_cities' is not defined

In [15]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    city_name=[]
    country_code=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},Texas,USA&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
#             print(response1)

        except:
            print("City not found. Skipping...")
            pass
        if response1["sys"]["country"]=="US":
            for x in unix_time():
                historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
                hist_json = requests.get(historical_url).json()
                temp.append(hist_json["current"]["temp"])
                pressure.append(hist_json["current"]["pressure"])
                wind_speed.append(hist_json["current"]["wind_speed"])
                city_name.append(response1['name'])
                country_code.append(response1["sys"]["country"])
    d=dict()
    d["City Name"] = city_name
    d["Country"] = country_code
    d["Temperature"] = temp
    d["Wind_Speed"] = wind_speed
    d["Pressure"] = pressure    
    return(d)
        #print ("--"*30)

In [16]:
df2 =pd.DataFrame(find_location(target_cities))

NameError: name 'target_cities' is not defined

In [ ]:
df2

In [ ]:
df[df["Country"]=="US"]